In [26]:
import pandas as pd
import numpy as np
import re
import time
import psutil
import shutil
import os
from transformers import pipeline
import json
import gzip
import bert_score
from torchmetrics.text.rouge import ROUGEScore
import glob
import wikipedia

In [27]:
def get_wiki_summary(url):
    
    if "wikipedia.org" not in url:
        return ""
    
    page_title = url.rpartition("/")[-1]
    page = wikipedia.page(title=page_title, auto_suggest=False)
    
    return page.summary
    
event_df = pd.read_json("CrisisFACTs-2022to2023.topics.json", lines=False).set_index("eventID")
event_df["wiki.summary"] = event_df["url"].apply(get_wiki_summary)

In [28]:
with open("final-annotated-facts-results.json", "r") as in_file:
    annotation_data = json.load(in_file)

all_req_ids = list(annotation_data.keys())

In [29]:
rouge = ROUGEScore(
    use_stemmer=True,
    rouge_keys=("rouge2",)
)

In [30]:
TOP_K = 32

for f in glob.glob("submissions/*.gz"):
    this_run_id = f.partition("/")[-1].replace(".gz", "")
    print(f, "-->", this_run_id)
    
    # Reset the summary for the current file
    local_event_summaries = {"CrisisFACTS-001": []}

    this_run_event_request_facts = {k: [] for k in all_req_ids}
    
    # Read and process the gzip file
    with gzip.open(f, "r") as in_file:
        for line_ in in_file:
            line = line_.decode("utf8")
            entry = json.loads(line)
            this_req_id = entry["requestID"]
            
            # Skip requests with no relevant facts
            if this_req_id not in all_req_ids:
                continue
            
            this_run_event_request_facts[this_req_id].append(entry)
    
    # Process facts only for 'CrisisFACTS-001'
    for event_request, this_fact_list in this_run_event_request_facts.items():
        event_id = event_request.rpartition("-")[0]

        if event_id != 'CrisisFACTS-001':
            continue

        sorted_fact_list = sorted(this_fact_list, key=lambda v: v["importance"], reverse=True)
        this_day_summary = [this_top_fact["factText"] for this_top_fact in sorted_fact_list[:TOP_K]]
        
        local_event_summaries["CrisisFACTS-001"] += this_day_summary
        
    # Compute metrics only for 'CrisisFACTS-001'
    event_id = "CrisisFACTS-001"
    this_submitted_summary = local_event_summaries[event_id]

    try:
        this_summary_text = ". ".join(this_submitted_summary).replace("..", ".")
    except Exception as e:
        print(f"Error processing summary for {event_id}: {e}")
        continue

    print(event_id, len(this_summary_text))
    
    # Retrieve the corresponding wiki summary
    wiki_summary = event_df.loc[event_id, "wiki.summary"]
    
    wiki_metric_bert_ = bert_score.score([this_summary_text], [wiki_summary], model_type="roberta-large-mnli")
    wiki_metric_rouge_ = rouge(this_summary_text, wiki_summary)
    
    wiki_metric_bert = {
        "precision": wiki_metric_bert_[0],
        "recall": wiki_metric_bert_[1],
        "f1": wiki_metric_bert_[2],
    }

    wiki_metric_rouge = {
        "precision": wiki_metric_rouge_["rouge2_precision"],
        "recall": wiki_metric_rouge_["rouge2_recall"],
        "f1": wiki_metric_rouge_["rouge2_fmeasure"],
    }
    

    this_wiki_df_bert = pd.DataFrame([{"metric": k, "value": v.item(), "event": event_id} for k, v in wiki_metric_bert.items()])
    this_wiki_df_rouge = pd.DataFrame([{"metric": k, "value": v.item(), "event": event_id} for k, v in wiki_metric_rouge.items()])

    print("Bert Score")
    print(this_wiki_df_bert)
    print("-------------------------------")
    print("Rouge Score")
    print(this_wiki_df_rouge)
    print("\n")

submissions/my_submission_pega_detail.json.gz --> my_submission_pega_detail.json
CrisisFACTS-001 48424
Bert Score
      metric     value            event
0  precision  0.717838  CrisisFACTS-001
1     recall  0.743328  CrisisFACTS-001
2         f1  0.730361  CrisisFACTS-001
-------------------------------
Rouge Score
      metric     value            event
0  precision  0.004859  CrisisFACTS-001
1     recall  0.291667  CrisisFACTS-001
2         f1  0.009558  CrisisFACTS-001


submissions/my_submission_gpt_detail.json.gz --> my_submission_gpt_detail.json
CrisisFACTS-001 10049
Bert Score
      metric     value            event
0  precision  0.695280  CrisisFACTS-001
1     recall  0.727899  CrisisFACTS-001
2         f1  0.711216  CrisisFACTS-001
-------------------------------
Rouge Score
      metric     value            event
0  precision  0.014393  CrisisFACTS-001
1     recall  0.145833  CrisisFACTS-001
2         f1  0.026201  CrisisFACTS-001


submissions/my_submission_bart_detail.json